In [81]:
import os
from sqlalchemy import create_engine
from langchain_community.utilities import SQLDatabase
from langchain_openai import ChatOpenAI
from langchain.agents import create_sql_agent
from langchain.agents.agent_types import AgentType
from dotenv import load_dotenv
from mysql import connector
load_dotenv(override=True)

True

In [82]:
openai_api_key = os.getenv("OPENAI_API_KEY")
if openai_api_key:
    print(f"OpenAI API Key exists")
else:
    print("OpenAI API Key not set")

OpenAI API Key exists


In [83]:
MYSQL_USER = os.getenv("DB_USER")
if MYSQL_USER:
    print(f"Database user exists as {MYSQL_USER}")
else: 
    print("no hay conexion")
MYSQL_PASSWORD = os.getenv("DB_PASSWORD")
MYSQL_HOST = os.getenv("DB_HOST") 
MYSQL_PORT = "3306"
MYSQL_DATABASE = os.getenv("DB_NAME")
MYSQL_DRIVER = "mysql+mysqlconnector"

DATABASE_URI = f"{MYSQL_DRIVER}://{MYSQL_USER}:{MYSQL_PASSWORD}@{MYSQL_HOST}:{MYSQL_PORT}/{MYSQL_DATABASE}"
print(f"conectado con la base {MYSQL_DATABASE}")

try:
    engine = create_engine(DATABASE_URI)
    db = SQLDatabase(engine)
    print("Conexión a MySQL exitosa y esquema de la base cargado.")

except Exception as e:
    print(f"Error al conectar a MySQL o inicializar la base de datos: {e}")
    exit()

Database user exists as compensaciones_rrhh
conectado con la base rrhh_app
Conexión a MySQL exitosa y esquema de la base cargado.


In [78]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
agent_executor = create_sql_agent(
    llm=llm,
    db=db,
    agent_type=AgentType.OPENAI_FUNCTIONS, 
    verbose=True # muestra el "pensamiento" del LLM 
)

In [80]:

def preguntar_a_la_base(pregunta):
    print(f"\n--- Pregunta del Usuario: {pregunta} ---")
    
    # El agente ejecuta el ciclo completo: piensa, genera SQL, ejecuta y responde
    try:
        respuesta = agent_executor.invoke({"input": pregunta})
        print("\n--- Respuesta Final del Chatbot ---")
        print(respuesta['output'])
    except Exception as e:
        print(f"\n--- Error en la ejecución del Agente ---")
        print(f"Ocurrió un error: {e}")
        print("El LLM puede haber generado un SQL incorrecto. Revisa el output con 'verbose=True'.")

# --- PRUEBAS ---
#preguntar_a_la_base("¿Cuántas personas con contrato indefinido estan activas y son operarios?")

# Pregunta que requiere agregar datos
preguntar_a_la_base("""Dame el número de empleados con status = 'activo', 
                    cuyo cargo actual (employees.name_role) contenga la palabra 'Operario', 
                    y que la jefa sea Nelsy Ariany Seijas Naranjo.""")

# Pregunta más compleja (puede requerir Joins si la BBDD es normalizada)
# El LLM intenta inferir la mejor manera de responder.
#preguntar_a_la_base("Muestra los nombres y correos electrónicos de los 5 empleados más jóvenes en el departamento de 'Ventas'.")


--- Pregunta del Usuario: Dame el número de empleados con status = 'activo', 
                    cuyo cargo actual (employees.name_role) contenga la palabra 'Operario', 
                    y que la jefa sea Nelsy Ariany Seijas Naranjo. ---


> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


absences, areas, consolidado_incidencias, contract_alerts, employees, employees_group, employees_peru, groups_table, historical_settlement_items, historical_settlements, job_history, licences, log_cargas_api, mail_corp, morpho_respaldo, permissions, position_level
Invoking: `sql_db_schema` with `{'table_names': 'employees, job_history'}`



CREATE TABLE employees (
	person_id INTEGER, 
	full_name VARCHAR(255), 
	first_name VARCHAR(100), 
	last_name VARCHAR(100), 
	rut VARCHAR(50) NOT NULL, 
	id INTEGER, 
	email VARCHAR(255), 
	personal_email VARCHAR(255), 
	address TEXT, 
	street VARCHAR(255), 
	street_number VARCHAR(50), 
	city VARCHAR(100), 
	province VARCHA